In [ ]:
#original with UTM coordinates
import rasterio
import numpy as np
from rasterio.plot import show
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker
from matplotlib.gridspec import GridSpec

# Set font globally to Times New Roman
plt.rcParams["font.family"] = "Times New Roman"

# Function to compute hillshade from DEM
def hillshade(array, azimuth=315, angle_altitude=45):
    x, y = np.gradient(array)
    slope = np.pi / 2.0 - np.arctan(np.sqrt(x*x + y*y))
    aspect = np.arctan2(-x, y)
    
    azimuth_rad = np.deg2rad(azimuth)
    altitude_rad = np.deg2rad(angle_altitude)
    
    shaded = np.sin(altitude_rad) * np.sin(slope) + \
             np.cos(altitude_rad) * np.cos(slope) * np.cos(azimuth_rad - aspect)
    
    return 255 * (shaded + 1) / 2  # Scale to 0-255 for better visualization

# Load the DEM raster (used for all subplots)
raster_path = "Dem_aggreg_15m.tif"
with rasterio.open(raster_path) as src:
    dem = src.read(1)  # Read the first band (DEM data)
    dem_bounds = src.bounds  # Get raster boundaries
    dem_nodata = src.nodata  # Retrieve the no-data value for the DEM

# Convert no-data values to NaN
dem = np.where(dem == dem_nodata, np.nan, dem)

# Compute hillshade from the DEM (this will be reused in all subplots)
hillshade_data = hillshade(dem)

# Prepare a list of shapefile paths for each subplot
shapefile_paths = [
    "Results_GIS/allProbs_terrain_CF.shp",  # Shapefile for subplot 1 row 1
    "Results_GIS/allProbs_terrain_CF.shp",  # Shapefile for subplot 2 row 1
    "Results_GIS/allProbs_terrain_allCum_CF.shp",  # Shapefile for subplot 3
    "Results_GIS/allProbs_terrain_allCum_CF.shp",  # Shapefile for subplot 4
    "Results_GIS/allProbs_daily_75_CF.shp",  # Shapefile for subplot 5
    "Results_GIS/allProbs_daily_75_CF.shp",  # Shapefile for subplot 6
    "Results_GIS/allProbs_hourly_CF.shp",    # Shapefile for subplot 7
    "Results_GIS/allProbs_hourly_CF.shp"     # Shapefile for subplot 8
]

# Use the same landslide susceptibility field ('EF') for all shapefiles
susc_column = 'EF'
cf_column = 'EF_CF'

# Titles for each subplot
subplot_titles = [ 
    "(a.1) Terrain only probability",
    "(a.2) Terrain only CF",
    "(b.1) Terrain + all scalar rainfall probability",
    "(b.2) Terrain + all scalar rainfall CF",
    "(c.1) Terrain + timeseries (daily) probability",
    "(c.2) Terrain + timeseries (daily) CF",
    "(d.1) Terrain + timeseries (hourly) probability",
    "(d.2) Terrain + timeseries (hourly) CF"
]

cf_colors = {
    'FN': 'firebrick',  # Dark Red
    'FP': 'lightcoral',  # Light Red
    'TN': 'lightgreen',  # Light Green
    'TP': 'darkgreen'    # Dark Green
}

# Function to save each row (pair of subplots) as separate TIFF files
def save_pair_as_tiff(row_idx):
    # Create a new figure for each row
    fig_row = plt.figure(figsize=(13, 10), dpi=500)  # Adjust figure size for each row
    gs_row = GridSpec(1, 2, figure=fig_row, wspace=0, hspace=0)
    
    for col in range(2):  # Two subplots in each row
        ax = fig_row.add_subplot(gs_row[col])
        
        i = row_idx * 2 + col  # Calculate the index for each subplot
        
        shapefile_path = shapefile_paths[i]
        
        # Load the landslide susceptibility shapefile
        shapefile_data = gpd.read_file(shapefile_path)

        # Plot the hillshade in grayscale
        ax.imshow(hillshade_data, cmap='gray', extent=[dem_bounds.left, dem_bounds.right, 
                                                       dem_bounds.bottom, dem_bounds.top],
                  zorder=1)

        # Get the bounds of the shapefile to use for the extent of the plot
        shape_bounds = shapefile_data.total_bounds  # This gives [xmin, ymin, xmax, ymax]

        # Adjust the limits of the plot to the bounds of the shapefile
        ax.set_xlim([shape_bounds[0], shape_bounds[2]])
        ax.set_ylim([shape_bounds[1], shape_bounds[3]])

        # Plot using different columns and color maps for different columns of the grid
        if i % 2 == 0:  # Left column: probability
            cmap = plt.get_cmap('RdYlBu_r')  # Divergent colormap for probability
            norm = mcolors.TwoSlopeNorm(vmin=0, vcenter=0.5, vmax=1)  # Set the range for EF
            shapefile_data.plot(column=susc_column, cmap=cmap, norm=norm, alpha=0.65, ax=ax, legend=False, zorder=2)
        else:  # Right column: CF (categorical)
            # Plot the categorical column with discrete colors, no need for 'column' argument
            shapefile_data.plot(ax=ax, color=shapefile_data[cf_column].map(cf_colors), alpha=0.7, legend=False, zorder=2)

        # Add the title inside each subplot at the top-right corner
        ax.text(0.98, 0.98, subplot_titles[i], transform=ax.transAxes, fontsize=12,
                verticalalignment='top', horizontalalignment='right', 
                bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

        # Remove y-axis labels and ticks for second column figures
        if col == 1:
            ax.set_yticklabels([])  # Remove y-axis labels
            ax.set_yticks([])       # Remove y-axis ticks

        # Remove x-axis ticks and labels for all rows except the last
        if row_idx < 3:
            ax.set_xticks([])  # Remove x-axis ticks
            ax.set_xticklabels([])  # Remove x-axis labels

        # Remove scientific notation (1e6) from all subplots
        ax.yaxis.get_offset_text().set_visible(False)
        ax.yaxis.set_major_formatter(mticker.ScalarFormatter(useOffset=False))

        # Apply scientific notation to x-axis on the last row only
        if row_idx == 3:
            ax.xaxis.set_major_formatter(mticker.ScalarFormatter(useOffset=True))
            ax.set_xticklabels([])  # Remove x-axis labels

    # Save the row as a TIFF with no padding/whitespace
    fig_row.savefig(f"EF_susc_CF_row_{row_idx + 1}.tif", format='tiff', bbox_inches='tight', pad_inches=0)
    plt.close(fig_row)

# Save each row (pair of subplots) as separate TIFF files
for row_idx in range(4):  # 4 rows total
    save_pair_as_tiff(row_idx)

# Generate and save the probability colorbar as a separate image with specified settings
fig_legend_prob, ax_legend_prob = plt.subplots(figsize=(8, 1), dpi=500)
sm_left = plt.cm.ScalarMappable(cmap=plt.get_cmap('RdYlBu_r'), norm=mcolors.TwoSlopeNorm(vmin=0, vcenter=0.5, vmax=1))
cbar_left = plt.colorbar(sm_left, ax=ax_legend_prob, orientation='horizontal')  # Use plt.colorbar
cbar_left.set_label('Landslide Probability', fontsize=14)
ax_legend_prob.axis('off')  # Hide the axis for a clean colorbar
fig_legend_prob.savefig("Probability_legend.tif", format='tiff', bbox_inches='tight', pad_inches=0)
plt.close(fig_legend_prob)

# Generate and save the CF legend as a separate image with no whitespace
fig_legend_cf, ax_legend_cf = plt.subplots(figsize=(8, 1), dpi=500)
legend_elements = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='darkgreen', markersize=12, label='TP (True Positive)'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='lightgreen', markersize=12, label='TN (True Negative)'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='lightcoral', markersize=12, label='FP (False Positive)'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='firebrick', markersize=12, label='FN (False Negative)'),
]
ax_legend_cf.axis('off')
fig_legend_cf.legend(handles=legend_elements, loc='center', ncol=2, columnspacing=1.5, frameon=True)
fig_legend_cf.savefig("CF_legend.tif", format='tiff', bbox_inches='tight', pad_inches=0)
plt.close(fig_legend_cf)


In [ ]:
# plot of rows with lat and long in degrees
import rasterio
import numpy as np
from rasterio.plot import show
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker
from pyproj import Transformer
from matplotlib.gridspec import GridSpec

# Set font globally to Times New Roman
plt.rcParams["font.family"] = "Times New Roman"

# Ensure output directory exists
output_dir = "Shapefile_Images"
#os.makedirs(output_dir, exist_ok=True)  # Creates the directory if it doesn't already exist

# Function to compute hillshade from DEM
def compute_hillshade(array, azimuth=315, altitude=45):
    x, y = np.gradient(array)
    slope = np.pi / 2.0 - np.arctan(np.sqrt(x * x + y * y))
    aspect = np.arctan2(-x, y)
    
    azimuth_rad = np.deg2rad(azimuth)
    altitude_rad = np.deg2rad(altitude)
    
    shaded = np.sin(altitude_rad) * np.sin(slope) + \
             np.cos(altitude_rad) * np.cos(slope) * np.cos(azimuth_rad - aspect)
    
    hillshade_normalized = 255 * (shaded + 1) / 2  # Scale to 0-255 for better visualization
    hillshade_normalized = np.where(np.isnan(hillshade_normalized), 255, hillshade_normalized)
    return hillshade_normalized.astype(np.uint8)

# Load the DEM raster and compute hillshade
raster_path = "Dem_aggreg_15m.tif"
with rasterio.open(raster_path) as src:
    dem = src.read(1)  # Read the first band (DEM data)
    dem = np.where(dem == src.nodata, np.nan, dem)  # Handle NoData values
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

# Compute realistic hillshade from DEM
hillshade_data = compute_hillshade(dem)

# Prepare a list of shapefile paths for each subplot
shapefile_paths = [
    "Results_GIS/allProbs_terrain_CF.shp",  # Shapefile for subplot 1 row 1
    "Results_GIS/allProbs_terrain_CF.shp",  # Shapefile for subplot 2 row 1
    "Results_GIS/allProbs_terrain_allCum_CF.shp",  # Shapefile for subplot 3
    "Results_GIS/allProbs_terrain_allCum_CF.shp",  # Shapefile for subplot 4
    "Results_GIS/allProbs_daily_75_CF.shp",  # Shapefile for subplot 5
    "Results_GIS/allProbs_daily_75_CF.shp",  # Shapefile for subplot 6
    "Results_GIS/allProbs_hourly_CF.shp",    # Shapefile for subplot 7
    "Results_GIS/allProbs_hourly_CF.shp"     # Shapefile for subplot 8
]

# Use the same landslide susceptibility field ('RS') for all shapefiles
susc_column = 'DS'
cf_column = 'DS_CF'

# Titles for each subplot
subplot_titles = [ 
    "(a.1) Terrain only probability",
    "(a.2) Terrain only CF",
    "(b.1) Terrain + all scalar rainfall probability",
    "(b.2) Terrain + all scalar rainfall CF",
    "(c.1) Terrain + timeseries (daily) probability",
    "(c.2) Terrain + timeseries (daily) CF",
    "(d.1) Terrain + timeseries (hourly) probability",
    "(d.2) Terrain + timeseries (hourly) CF"
]

cf_colors = {
    'FN': 'firebrick',  # Dark Red
    'FP': 'lightcoral',  # Light Red
    'TN': 'lightgreen',  # Light Green
    'TP': 'darkgreen'    # Dark Green
}


# Function to save each row (pair of subplots) as separate TIFF files
def save_pair_as_tiff(row_idx):
    # Create a new figure for each row
    fig_row = plt.figure(figsize=(13, 10), dpi=500)  # Adjust figure size for each row
    gs_row = GridSpec(1, 2, figure=fig_row, wspace=0, hspace=0)
    
    for col in range(2):  # Two subplots in each row
        ax = fig_row.add_subplot(gs_row[col])
        
        i = row_idx * 2 + col  # Calculate the index for each subplot
        
        shapefile_path = shapefile_paths[i]
        
        # Load the landslide susceptibility shapefile
        shapefile_data = gpd.read_file(shapefile_path)

        # Reproject the shapefile to lat/lon (EPSG:4326)
        shapefile_data = shapefile_data.to_crs(epsg=4326)

        # Plot the hillshade in grayscale
        ax.imshow(hillshade_data, cmap='gray', extent=[lon_min, lon_max, lat_min, lat_max],
                  zorder=1)

        # Adjust the limits of the plot to the bounds of the shapefile
        ax.set_xlim([lon_min, lon_max])
        ax.set_ylim([lat_min, lat_max])

        # Plot using different columns and color maps for different columns of the grid
        if i % 2 == 0:  # Left column: probability
            cmap = plt.get_cmap('RdYlBu_r')  # Divergent colormap for probability
            norm = mcolors.TwoSlopeNorm(vmin=0, vcenter=0.5, vmax=1)  # Set the range for RS
            shapefile_data.plot(column=susc_column, cmap=cmap, norm=norm, alpha=0.65, ax=ax, legend=False, zorder=2)
        else:  # Right column: CF (categorical)
            shapefile_data.plot(ax=ax, color=shapefile_data[cf_column].map(cf_colors), alpha=0.7, legend=False, zorder=2)

        # Add the title inside each subplot at the top-right corner
        ax.text(0.98, 0.98, subplot_titles[i], transform=ax.transAxes, fontsize=12,
                verticalalignment='top', horizontalalignment='right', 
                bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

        # Add axis ticks in degrees without labels
        if col == 0:  # Y-axis ticks for the first column (all rows)
            ax.yaxis.set_major_locator(mticker.MultipleLocator(1/3))  # 20-minute intervals
            ax.yaxis.set_major_formatter(mticker.FuncFormatter(format_degrees))
            ax.tick_params(axis='y', which='both', direction='out', length=5, width=0.5, colors='black', labelleft=False)
        else:
            ax.set_yticks([])  # Remove y-axis ticks for other columns
        
        if row_idx == 3:  # X-axis ticks only for last row, no labels
            ax.xaxis.set_major_locator(mticker.MultipleLocator(1/2))  # 30-minute intervals
            ax.xaxis.set_major_formatter(mticker.FuncFormatter(format_degrees))
            ax.tick_params(axis='x', which='both', direction='out', length=5, width=0.5, colors='black', labelbottom=False)
            # ax.yaxis.set_major_locator(mticker.MultipleLocator(1/3))  # 20-minute intervals
            # ax.yaxis.set_major_formatter(mticker.FuncFormatter(format_degrees))
            # ax.tick_params(axis='y', which='both', direction='out', length=5, width=0.5, colors='black', labelleft=False)
        else:
            ax.set_xticks([])  # Remove x-axis ticks for other rows

        # Remove scientific notation (1e6) from all subplots
        ax.yaxis.get_offset_text().set_visible(False)
        ax.yaxis.set_major_formatter(mticker.ScalarFormatter(useOffset=False))

    # Save the row as a TIFF with no padding/whitespace
    fig_row.savefig(f"Plots/RS_susc_CF_row_{row_idx + 1}.tif", format='tiff', bbox_inches='tight', pad_inches=0)
    plt.close(fig_row)

# Save each row (pair of subplots) as separate TIFF files
for row_idx in range(4):  # 4 rows total
    save_pair_as_tiff(row_idx)

# Generate and save the probability colorbar as a separate image with specified settings
fig_legend_prob, ax_legend_prob = plt.subplots(figsize=(8, 1), dpi=500)
sm_left = plt.cm.ScalarMappable(cmap=plt.get_cmap('RdYlBu_r'), norm=mcolors.TwoSlopeNorm(vmin=0, vcenter=0.5, vmax=1))
cbar_left = plt.colorbar(sm_left, ax=ax_legend_prob, orientation='horizontal')  # Use plt.colorbar
cbar_left.set_label('Landslide Probability', fontsize=14)
ax_legend_prob.axis('off')  # Hide the axis for a clean colorbar
fig_legend_prob.savefig("Probability_legend.tif", format='tiff', bbox_inches='tight', pad_inches=0)
plt.close(fig_legend_prob)

# Generate and save the CF legend as a separate image with no whitespace
fig_legend_cf, ax_legend_cf = plt.subplots(figsize=(8, 1), dpi=500)
legend_elements = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='darkgreen', markersize=12, label='TP (True Positive)'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='lightgreen', markersize=12, label='TN (True Negative)'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='lightcoral', markersize=12, label='FP (False Positive)'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='firebrick', markersize=12, label='FN (False Negative)'),
]
ax_legend_cf.axis('off')
fig_legend_cf.legend(handles=legend_elements, loc='center', ncol=2, columnspacing=1.5, frameon=True)
fig_legend_cf.savefig("CF_legend.tif", format='tiff', bbox_inches='tight', pad_inches=0)
plt.close(fig_legend_cf)


In [ ]:
# zoomed parts
import rasterio
import numpy as np
from rasterio.plot import show
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from pyproj import Transformer

# Set font globally to Times New Roman
plt.rcParams["font.family"] = "Times New Roman"

# Function to compute hillshade from DEM
def hillshade(array, azimuth=315, angle_altitude=45):
    x, y = np.gradient(array)
    slope = np.pi / 2.0 - np.arctan(np.sqrt(x*x + y*y))
    aspect = np.arctan2(-x, y)
    
    azimuth_rad = np.deg2rad(azimuth)
    altitude_rad = np.deg2rad(angle_altitude)
    
    shaded = np.sin(altitude_rad) * np.sin(slope) + \
             np.cos(altitude_rad) * np.cos(slope) * np.cos(azimuth_rad - aspect)
    
    return 255 * (shaded + 1) / 2  # Scale to 0-255 for better visualization

# Load the DEM raster (used for all subplots)
raster_path = "Dem_aggreg_15m.tif"
with rasterio.open(raster_path) as src:
    dem = src.read(1)  # Read the first band (DEM data)
    dem_bounds = src.bounds  # Get raster boundaries
    dem_nodata = src.nodata  # Retrieve the no-data value for the DEM

# Convert no-data values to NaN
dem = np.where(dem == dem_nodata, np.nan, dem)

# Convert DEM bounds from UTM to lat/lon using Transformer
transformer = Transformer.from_crs(src.crs, "EPSG:4326", always_xy=True)
lon_min, lat_min = transformer.transform(dem_bounds.left, dem_bounds.bottom)
lon_max, lat_max = transformer.transform(dem_bounds.right, dem_bounds.top)

# Zoomed area in lat/lon (define the specific coordinates for the zoom)
lon_min_zoom, lon_max_zoom = 11.58, 11.85  # Example: specify your zoomed longitude bounds
lat_min_zoom, lat_max_zoom = 44.18, 44.28  # Example: specify your zoomed latitude bounds

# Compute hillshade from the DEM (this will be reused in all subplots)
hillshade_data = hillshade(dem)

# Prepare a list of shapefile paths for each subplot
shapefile_paths = [
    "Results_GIS/allProbs_terrain_CF.shp",  # Shapefile for subplot 1
    "Results_GIS/allProbs_terrain_CF.shp",  # Shapefile for subplot 2
    "Results_GIS/allProbs_terrain_allCum_CF.shp",  # Shapefile for subplot 3
    "Results_GIS/allProbs_terrain_allCum_CF.shp",  # Shapefile for subplot 4
    "Results_GIS/allProbs_daily_75_CF.shp",  # Shapefile for subplot 5
    "Results_GIS/allProbs_daily_75_CF.shp",  # Shapefile for subplot 6
    "Results_GIS/allProbs_hourly_CF.shp",    # Shapefile for subplot 7
    "Results_GIS/allProbs_hourly_CF.shp"     # Shapefile for subplot 8
]

# Use the same landslide susceptibility field ('EF') for all shapefiles
susc_column = 'EF'
cf_column = 'EF_CF'

cf_colors = {
    'FN': 'firebrick',  # Dark Red
    'FP': 'lightcoral',  # Light Red
    'TN': 'lightgreen',  # Light Green
    'TP': 'darkgreen'    # Dark Green
}

# Function to save each image as a separate TIFF file
def save_zoomed_image(image_idx):
    fig, ax = plt.subplots(figsize=(5, 3), dpi=600)

    shapefile_path = shapefile_paths[image_idx]
    
    # Load the landslide susceptibility shapefile
    shapefile_data = gpd.read_file(shapefile_path)

    # Reproject the shapefile to lat/lon (EPSG:4326)
    shapefile_data = shapefile_data.to_crs(epsg=4326)

    # Plot the hillshade in grayscale
    ax.imshow(hillshade_data, cmap='gray', extent=[lon_min, lon_max, lat_min, lat_max],
              zorder=1)

    # Adjust the limits of the plot to the zoomed area
    ax.set_xlim([lon_min_zoom, lon_max_zoom])
    ax.set_ylim([lat_min_zoom, lat_max_zoom])

    # Plot using different columns and color maps for different columns of the grid
    if image_idx % 2 == 0:  # Probability maps (left column)
        cmap = plt.get_cmap('RdYlBu_r')  # Divergent colormap for probability
        norm = mcolors.TwoSlopeNorm(vmin=0, vcenter=0.5, vmax=1)  # Set the range for EF
        shapefile_data.plot(column=susc_column, cmap=cmap, norm=norm, alpha=0.65, ax=ax, legend=False, zorder=2)
    else:  # CF maps (right column)
        shapefile_data.plot(ax=ax, color=shapefile_data[cf_column].map(cf_colors), alpha=0.7, legend=False, zorder=2)

    # Remove all axis ticks and labels
    ax.set_xticks([])
    ax.set_yticks([])
    
    # Save the plot as TIFF
    fig.savefig(f"PLots/Zoomed_EF_{image_idx + 1}.tif", format='tiff', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

# Save each image as a separate TIFF file for each shapefile
for image_idx in range(len(shapefile_paths)):
    save_zoomed_image(image_idx)


In [1]:
# GOOD ONE WITH UTM COOR 
import os
import numpy as np
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker
from rasterio.plot import show
from matplotlib.gridspec import GridSpec

# Ensure output directory exists
output_dir = "Shapefile_Images"
# os.makedirs(output_dir, exist_ok=True)  # Uncomment to create output directory if needed

# Function to compute hillshade from DEM
def compute_hillshade(array, azimuth=315, altitude=45):
    """
    Compute hillshade visualization for a DEM.
    
    Parameters:
        array (numpy.ndarray): DEM array.
        azimuth (float): Azimuth angle for hillshade.
        altitude (float): Altitude angle for hillshade.
    
    Returns:
        numpy.ndarray: Hillshade visualization scaled between 0-255.
    """
    x, y = np.gradient(array)
    slope = np.pi / 2.0 - np.arctan(np.sqrt(x * x + y * y))
    aspect = np.arctan2(-x, y)
    
    azimuth_rad = np.deg2rad(azimuth)
    altitude_rad = np.deg2rad(altitude)
    
    shaded = np.sin(altitude_rad) * np.sin(slope) + \
             np.cos(altitude_rad) * np.cos(slope) * np.cos(azimuth_rad - aspect)
    
    hillshade_normalized = 255 * (shaded + 1) / 2  # Scale to 0-255 for visualization
    hillshade_normalized = np.where(np.isnan(hillshade_normalized), 255, hillshade_normalized)
    return hillshade_normalized.astype(np.uint8)

# Load the DEM raster and compute hillshade
raster_path = "Dem_aggreg_15m.tif"
with rasterio.open(raster_path) as src:
    dem = src.read(1)  # Read the first band (DEM data)
    dem = np.where(dem == src.nodata, np.nan, dem)  # Handle NoData values
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

# Compute hillshade from DEM
hillshade_data = compute_hillshade(dem)

# Load shapefiles for analysis
study_area_path = "Study_area.shp"
study_area = gpd.read_file(study_area_path)  # Study area shapefile

# Get the bounds of the study area to use for the extent of the plot
shape_bounds = study_area.total_bounds  # This gives [xmin, ymin, xmax, ymax]

shapefile_paths = [
    "Results_GIS/allProbs_terrain_CF.shp",
    "Results_GIS/allProbs_terrain_CF.shp",
    "Results_GIS/allProbs_terrain_allCum_CF.shp",
    "Results_GIS/allProbs_terrain_allCum_CF.shp",
    "Results_GIS/allProbs_daily_75_CF.shp",
    "Results_GIS/allProbs_daily_75_CF.shp",
    "Results_GIS/allProbs_hourly_CF.shp",
    "Results_GIS/allProbs_hourly_CF.shp"
]

# Columns for susceptibility and confusion matrix
susc_column = 'RS'
cf_column = 'RS_CF'

# # Titles for subplots
# subplot_titles = [
#     "(a.1) Terrain only probability",
#     "(a.2) Terrain only CF",
#     "(b.1) Terrain + all scalar rainfall probability",
#     "(b.2) Terrain + all scalar rainfall CF",
#     "(c.1) Terrain + timeseries (daily) probability",
#     "(c.2) Terrain + timeseries (daily) CF",
#     "(d.1) Terrain + timeseries (hourly) probability",
#     "(d.2) Terrain + timeseries (hourly) CF"
# ]

# Titles for subplots
subplot_titles = [
    "a.1)",
    "a.2)",
    "b.1)",
    "b.2)",
    "c.1)",
    "c.2)",
    "d.1)",
    "d.2)"
]

# Colors for confusion matrix categories
cf_colors = {
    'FN': 'firebrick',   # False Negative - Dark Red
    'FP': 'lightcoral',  # False Positive - Light Red
    'TN': 'lightgreen',  # True Negative - Light Green
    'TP': 'darkgreen'    # True Positive - Dark Green
}

# Function to save each row (pair of subplots) as separate TIFF files
def save_pair_as_tiff(row_idx):
    """
    Save a pair of subplots for a given row as a TIFF file.
    
    Parameters:
        row_idx (int): Row index (0-based) indicating which row of subplots to save.
    """
    fig_row = plt.figure(figsize=(15, 10), dpi=600)  # Adjust figure size
    gs_row = GridSpec(1, 2, figure=fig_row, wspace=0, hspace=0)  # Define subplot grid
    
    for col in range(2):  # Two subplots per row
        ax = fig_row.add_subplot(gs_row[col])
        i = row_idx * 2 + col  # Calculate subplot index
        
        shapefile_path = shapefile_paths[i]
        shapefile_data = gpd.read_file(shapefile_path)  # Load shapefile

        # Plot hillshade background
        ax.imshow(hillshade_data, extent=extent, cmap='gray', zorder=1)

        # Adjust plot extent to match the study area
        ax.set_xlim([shape_bounds[0], shape_bounds[2]])
        ax.set_ylim([shape_bounds[1], shape_bounds[3]])

        # Plot susceptibility or confusion matrix data
        if i % 2 == 0:  # Left column: Probability map
            cmap = plt.get_cmap('RdYlBu_r')  # Divergent colormap
            norm = mcolors.TwoSlopeNorm(vmin=0, vcenter=0.5, vmax=1)  # Normalize values
            shapefile_data.plot(column=susc_column, cmap=cmap, norm=norm, alpha=0.7, ax=ax, legend=False, zorder=2)
        else:  # Right column: Confusion matrix map
            shapefile_data.plot(ax=ax, color=shapefile_data[cf_column].map(cf_colors), alpha=0.7, legend=False, zorder=2)

        # Plot study area boundary in red
        study_area.boundary.plot(ax=ax, color='red', linewidth=0.5, zorder=3)

        # Add title inside subplot
        ax.text(0.5, 0.95, subplot_titles[i], transform=ax.transAxes, fontsize=16,
                verticalalignment='top', horizontalalignment='center',
                bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))
        
        # # Customize axis ticks
        # if col == 0:  # Y-axis ticks for the first column
        #     ax.yaxis.set_major_locator(mticker.MultipleLocator(50000))  # 50,000-meter intervals
        #     ax.tick_params(axis='y', which='major', direction='out', length=5, width=0.5, colors='black', labelleft=False)
        # else:
        #     ax.set_yticks([])  # Remove Y-axis ticks for other columns

        # if row_idx == 3:  # X-axis ticks only for the last row
        #     ax.xaxis.set_major_locator(mticker.MultipleLocator(5 0000))  # 50,000-meter intervals
        #     ax.tick_params(axis='x', which='major', direction='out', length=5, width=0.5, colors='black', labelbottom=False)
        # else:
        #     ax.set_xticks([])  # Remove X-axis ticks for other rows

        # Remove all ticks and labels
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xticklabels([])
        ax.set_yticklabels([])

    # Save row as TIFF
    fig_row.savefig(f"Plots/RS_new_susc_CF_row_{row_idx + 1}.tif", format='tiff', bbox_inches='tight', pad_inches=0)
    plt.close(fig_row)

# Save each row (pair of subplots)
for row_idx in range(4):  # Four rows in total
    save_pair_as_tiff(row_idx)

# # Save probability legend
# fig_legend_prob, ax_legend_prob = plt.subplots(figsize=(8, 1), dpi=600)
# sm_left = plt.cm.ScalarMappable(cmap=plt.get_cmap('RdYlBu_r'), norm=mcolors.TwoSlopeNorm(vmin=0, vcenter=0.5, vmax=1))
# cbar_left = plt.colorbar(sm_left, ax=ax_legend_prob, orientation='horizontal')
# cbar_left.set_label('Landslide Probability', fontsize=14)
# ax_legend_prob.axis('off')
# fig_legend_prob.savefig("Probability_legend.tif", format='tiff', bbox_inches='tight', pad_inches=0)
# plt.close(fig_legend_prob)

# # Save confusion matrix legend
# fig_legend_cf, ax_legend_cf = plt.subplots(figsize=(8, 1), dpi=500)
# legend_elements = [
#     plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='darkgreen', markersize=12, label='TP (True Positive)'),
#     plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='lightgreen', markersize=12, label='TN (True Negative)'),
#     plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='lightcoral', markersize=12, label='FP (False Positive)'),
#     plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='firebrick', markersize=12, label='FN (False Negative)'),
# ]
# ax_legend_cf.axis('off')
# fig_legend_cf.legend(handles=legend_elements, loc='center', ncol=2, columnspacing=1.5, frameon=True)
# fig_legend_cf.savefig("CF_legend.tif", format='tiff', bbox_inches='tight', pad_inches=0)

#plt.close(fig_legend_cf)
